In [2]:
import numpy as np
import pandas as pd
from quantum_clustering import QKMeans, QFCMeans
from qiskit import BasicAer
from qiskit.utils import QuantumInstance, algorithm_globals
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from fcmeans import FCM

seed = 42
algorithm_globals.random_seed = seed
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                    shots=1024,
                                    optimization_level=1,
                                    seed_simulator=seed,
                                    seed_transpiler=seed)

In [3]:
# Use iris data set for training and test data
X, y = load_iris(return_X_y=True)
num_features = 2
X = np.asarray([x[0:num_features] for x, y_ in zip(X, y) if y_ != 2])
y = np.asarray([y_ for x, y_ in zip(X, y) if y_ != 2])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [4]:
# fit K-means
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_train)
kmeans_labels = kmeans.labels_

# fit fuzzy C-means
fcmeans = FCM(n_clusters=3)
fcmeans.fit(X_train)
fcmeans_labels = fcmeans.predict(X_train)

# fit quantum K-means
qkmeans = QKMeans(n_clusters=3,
                    quantum_instance=quantum_instance
)
qkmeans.fit(X_train)
qkmeans_labels = qkmeans.labels_

# fit quantum fuzzy C-means
qfcmeans = QFCMeans(n_clusters=3,
                    quantum_instance=quantum_instance
)
qfcmeans.fit(X_train)
qfcm_labels = (qfcmeans.labels_)

In [5]:
# valiDATE
kmeans_predictions = kmeans.predict(X_test)
fcmeans_predictions = fcmeans.predict(X_test)
qkmeans_predictions = qkmeans.predict(X_test)
qfcmeans_predictions = qfcmeans.predict(X_test)

In [6]:
data = {"original_labels": y_train, 
           "kmeans_labels": kmeans_labels,
           "fcmeans_labels": fcmeans_labels, 
           "qkmeans_labels": qkmeans_labels,
           "qfcm_labels": qfcm_labels}
df = pd.DataFrame(data=data)
df.head(20)

,original_labels,kmeans_labels,fcmeans_labels,qkmeans_labels,qfcm_labels
0,1,1,0,1,2
1,1,1,0,1,2
2,0,0,1,0,0
3,0,0,1,0,0
4,1,1,0,1,2
5,0,0,1,0,1
6,0,0,1,0,1
7,1,1,0,1,2
8,0,0,1,0,0
9,1,2,2,1,2


In [7]:
qfcm_labels_new = np.where(qfcm_labels == 1, 2, qfcm_labels)
qfcm_labels_new = np.where(qfcm_labels == 2, 1, qfcm_labels_new)

qfcmeans_predictions_new = np.where(qfcmeans_predictions == 1, 2, qfcmeans_predictions)
qfcmeans_predictions_new = np.where(qfcmeans_predictions == 2, 1, qfcmeans_predictions_new)

In [8]:
data_train = {"original_labels": y_train, 
           "kmeans_labels": kmeans_labels,
           "fcmeans_labels": fcmeans_labels, 
           "qkmeans_labels": qkmeans_labels,
           "qfcm_labels": qfcm_labels_new}
df_train = pd.DataFrame(data=data)
df_train.head(5)

,original_labels,kmeans_labels,fcmeans_labels,qkmeans_labels,qfcm_labels
0,1,1,0,1,2
1,1,1,0,1,2
2,0,0,1,0,0
3,0,0,1,0,0
4,1,1,0,1,2


In [9]:
data_test = {"original_labels": y_test, 
        "kmeans_labels": kmeans_predictions,
        "fcmeans_labels": fcmeans_predictions, 
        "qkmeans_labels": qkmeans_predictions,
        "qfcm_labels": qfcmeans_predictions_new}
df_test = pd.DataFrame(data=data_test)
df_test.head(5)

,original_labels,kmeans_labels,fcmeans_labels,qkmeans_labels,qfcm_labels
0,1,2,0,1,1
1,1,1,0,1,1
2,1,2,2,2,1
3,0,0,1,0,0
4,0,0,1,0,2


In [10]:
# K-means
print('K-means: ', accuracy_score(y_train, kmeans_labels))

# fuzzy C-means
print('fuzzy C-means: ', accuracy_score(y_train, fcmeans_labels))

# quantum K-means
print('quantum K-means: ', accuracy_score(y_train, qkmeans_labels))

# quantum fuzzy C-means
print('quantum fuzzy C-means: ', accuracy_score(y_train, qfcm_labels_new))

K-means:  0.7375
fuzzy C-means:  0.0
quantum K-means:  0.775
quantum fuzzy C-means:  0.725


In [11]:
print('K-means')
print(classification_report(y_train, qkmeans_labels, labels=[0, 1, 2]))

K-means
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        38
           1       0.96      0.60      0.74        42
           2       0.00      0.00      0.00         0

    accuracy                           0.78        80
   macro avg       0.65      0.52      0.57        80
weighted avg       0.98      0.78      0.85        80



/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa

In [12]:
print('fuzzy C-means')
print(classification_report(y_train, fcmeans_labels, labels=[0, 1, 2]))

fuzzy C-means
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      38.0
           1       0.00      0.00      0.00      42.0
           2       0.00      0.00      0.00       0.0

    accuracy                           0.00      80.0
   macro avg       0.00      0.00      0.00      80.0
weighted avg       0.00      0.00      0.00      80.0



/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa

In [13]:
print('quantum K-means')
print(classification_report(y_train, qkmeans_labels, labels=[0, 1, 2]))

quantum K-means
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        38
           1       0.96      0.60      0.74        42
           2       0.00      0.00      0.00         0

    accuracy                           0.78        80
   macro avg       0.65      0.52      0.57        80
weighted avg       0.98      0.78      0.85        80



/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa

In [14]:
print('quantum fuzzy C-means')
print(classification_report(y_train, qfcm_labels_new, labels=[0, 1, 2]))

quantum fuzzy C-means
              precision    recall  f1-score   support

           0       0.84      0.55      0.67        38
           1       1.00      0.88      0.94        42
           2       0.00      0.00      0.00         0

    accuracy                           0.73        80
   macro avg       0.61      0.48      0.53        80
weighted avg       0.92      0.72      0.81        80



/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofia/Documents/quantum-computing-adjustments/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa

In [15]:
# K-means
print('K-means: ', accuracy_score(y_test, kmeans_predictions))

# fuzzy C-means
print('fuzzy C-means: ', accuracy_score(y_test, fcmeans_predictions))

# quantum K-means
print('quantum K-means: ', accuracy_score(y_test, qkmeans_predictions))

# quantum fuzzy C-means
print('quantum fuzzy C-means: ', accuracy_score(y_test, qfcmeans_predictions_new))

K-means:  0.75
fuzzy C-means:  0.0
quantum K-means:  0.8
quantum fuzzy C-means:  0.55
